In [2]:
import sys
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git")
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git/extern/nes-async-rl/")

In [3]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper

import numpy as np
import matplotlib.pyplot as plt
import itertools
import os.path

In [4]:
from dqn_phi import dqn_phi
import nes 
from a3c_nes import A3CLSTM
import chainer
from chainer import serializers

In [5]:
example_model_path = "/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5" 
rom_path = "/home/mathia/mario.nes"
nr_of_attempts = 30

In [22]:
base_paht = "/home/mathia/checkpoints/{}/{}"
model_names = [
    "scaled_fine_main_reward_1"
    ,"nes_a3c_4"
    ,"nes_a3c_scaled_finer_main_reward_1"
    ,"scaled_finer_main_reward_with_points_1"
]
#model_names = ["nes_a3c_scaled_finer_main_reward_1"]
model_chepoint_combos = [(model_name,"16000000_finish.h5")for model_name in model_names]
# model_chepoint_combos.append(("nes_a3c_4","9000000.h5"))
# model_chepoint_combos.append(("scaled_fine_main_reward_1","9400000.h5"))
# model_chepoint_combos.append(("nes_a3c_scaled_finer_main_reward_1","9200000.h5"))
#model_chepoint_combos = itertools.product(model_names, chepoint_filenames) 
model_snapshots = {(directory,checkpoint):base_paht.format(directory,checkpoint) 
                   for (directory,checkpoint) 
                   in model_chepoint_combos}
# make sure every file exists
for path in model_snapshots.values():
    if not os.path.isfile(path):
        print(path)

In [6]:
model_snapshots

{('nes_a3c_4',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_4/16000000_finish.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5',
 ('scaled_fine_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5',
 ('scaled_finer_main_reward_with_points_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_finer_main_reward_with_points_1/16000000_finish.h5'}

In [12]:
#n_actions = nes.NES(hsa.machine_constants.mario_rom_location).number_of_actions

def load_model(weights_path):
    n_actions = nes.n_actions
    model = A3CLSTM(n_actions)
    serializers.load_hdf5(weights_path, model)
    return model
example_model = load_model(example_model_path)

In [17]:
def make_eval_functions():
    return {
        "main_reward": rewards.make_finer_main_reward(),
        "points": rewards.make_delta_points(),
        "time_left": make_time_left()
    }

In [18]:
def eval_performance(model, nesApi, env, deterministic=False):
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        ehrenbrav_sum += env.receive_action(a)
        
        # TODO Sum of Ehrenbrav, Sum of Main Reward, sum of time left, did it beat the level?, how many deaths
        ram = nesApi.getRAM()
        for name, reward_function in other_rewards.items():
            sums[name] += reward_function(ram)
    
    sums["ehrenbrav"] = ehrenbrav_sum
    return sums

In [38]:
eval_performance(hsa.machine_constants.mario_rom_location,example_model,deterministic=False)

{'ehrenbrav': 14405, 'main_reward': 7.890625, 'points': 40.0, 'time_left': 0}

In [23]:
eval_results_d = {}
nesApi = NESInterface(rom_path,auto_render_period=2)
nesApi.render()
for (experiment_name,model_name), weights in model_snapshots.items():
    model = load_model(weights)
    for attempt_nr in range(nr_of_attempts):
        nesApi.reset_game()
        env = nes.NES(rom_path,outside_nes_interface=nesApi)
        performance = eval_performance(model,nesApi,env,deterministic=False)
        print((experiment_name,model_name,attempt_nr), performance)
        eval_results_d[(experiment_name,model_name,attempt_nr)] = performance

('scaled_fine_main_reward_1', '16000000_finish.h5', 0) {'ehrenbrav': 9950, 'main_reward': 6.046875, 'time_left': 0, 'points': 10.0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 1) {'ehrenbrav': 12215, 'main_reward': 5.96484375, 'time_left': 0, 'points': 30.0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 2) {'ehrenbrav': 11090, 'main_reward': 6.99609375, 'time_left': 0, 'points': 30.0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 3) {'ehrenbrav': 12550, 'main_reward': 7.89453125, 'time_left': 0, 'points': 60.0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 4) {'ehrenbrav': 9660, 'main_reward': 6.078125, 'time_left': 0, 'points': 10.0}


...

('nes_a3c_4', '16000000_finish.h5', 25) {'ehrenbrav': 11070, 'main_reward': 2.63671875, 'time_left': 0, 'points': 50.0}


('nes_a3c_4', '16000000_finish.h5', 26) {'ehrenbrav': 13865, 'main_reward': 4.8671875, 'time_left': 0, 'points': 50.0}


('nes_a3c_4', '16000000_finish.h5', 27) {'ehrenbrav': 14730, 'main_reward': 6.046875, 'time_left': 0, 'points': 100.0}


('nes_a3c_4', '16000000_finish.h5', 28) {'ehrenbrav': 9675, 'main_reward': 5.92578125, 'time_left': 0, 'points': 10.0}


('nes_a3c_4', '16000000_finish.h5', 29) {'ehrenbrav': 20220, 'main_reward': 5.99609375, 'time_left': 0, 'points': 80.0}


In [24]:
result_d_df = pandas.DataFrame.from_dict(eval_results_d,orient="index")

In [25]:
result_d_df.to_csv("eval_results_deterministisch.csv")

In [26]:
result_d_df.to_pickle("eval_results_deterministisch.pickle")

In [19]:
result_df.groupby?

# ladida just rewiting time left

In [16]:
def make_time_left():
    counted_once = False
    def time_left(ram):
        nonlocal counted_once
        if counted_once:
            return 0
        if ram[0x001D] == 3:
            counted_once = True
            return rewards.time_left(ram) 
        return 0
    return  time_left

## Forgot the small main reward 